In [1]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/fracture_detection/MURA-v1.1.zip"
extract_path = "/content/"
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Unzipping completed!")

Unzipping completed!


In [11]:
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from glob import glob
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import copy


Data Augmentation

In [12]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [13]:
!pip install -q timm

In [14]:
import pandas as pd

def load_paths_labels_from_csv(csv_file):
    df = pd.read_csv(csv_file, header=None)
    image_paths = []
    labels = []

    for i, row in df.iterrows():
        rel_path = row[0]
        label = 1 if "positive" in rel_path.lower() else 0
        full_path = os.path.join("/content", rel_path)

        if os.path.isfile(full_path):
            image_paths.append(full_path)
            labels.append(label)

    return image_paths, labels



In [15]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from glob import glob
import torch
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import numpy as np


class MURADataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        if self.transform:
            image = self.transform(image)
        return image, label

def load_paths_labels(base_dir):
    image_paths, labels = [], []
    for phase in ["train", "valid"]:
        phase_dir = os.path.join(base_dir, phase)
        for body_part in os.listdir(phase_dir):
            body_part_dir = os.path.join(phase_dir, body_part)
            if not os.path.isdir(body_part_dir):
                continue
            for patient in os.listdir(body_part_dir):
                patient_dir = os.path.join(body_part_dir, patient)
                if not os.path.isdir(patient_dir):
                    continue
                for study in os.listdir(patient_dir):
                    study_dir = os.path.join(patient_dir, study)
                    if not os.path.isdir(study_dir):
                        continue
                    label = 1 if "positive" in study.lower() else 0
                    for img in glob(os.path.join(study_dir, "*.png")):
                        image_paths.append(img)
                        labels.append(label)
    return image_paths, labels


image_paths, labels = load_paths_labels("/content/MURA-v1.1")
labels = np.array(labels)

# 5-fold cross-validation setup
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(image_paths)):
    print(f"\n--- Fold {fold+1}/{k_folds} ---")

    train_paths = [image_paths[i] for i in train_idx]
    val_paths = [image_paths[i] for i in val_idx]
    train_labels = labels[train_idx]
    val_labels = labels[val_idx]

    train_dataset = MURADataset(train_paths, train_labels, transform=train_transform)
    val_dataset = MURADataset(val_paths, val_labels, transform=val_transform)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)





--- Fold 1/5 ---

--- Fold 2/5 ---

--- Fold 3/5 ---

--- Fold 4/5 ---

--- Fold 5/5 ---


In [16]:
class CNNTransformerModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = torchvision.models.resnet50(pretrained=True)
        self.cnn.fc = nn.Identity()

        self.fc_proj = nn.Linear(2048, 768)

        encoder_layer = nn.TransformerEncoderLayer(d_model=768, nhead=8)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=1)

        self.classifier = nn.Linear(768, 1)

    def forward(self, x):
        x = self.cnn(x)
        x = self.fc_proj(x)
        x = x.unsqueeze(1)
        x = self.transformer(x)
        x = x.squeeze(1)
        x = self.classifier(x)
        return x


In [17]:
import warnings
warnings.filterwarnings("ignore", message="The secret `HF_TOKEN` does not exist")


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNTransformerModel().to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

best_model_wts = copy.deepcopy(model.state_dict())
best_f1 = 0
patience, patience_counter = 3, 0

def train_one_epoch():
    model.train()
    running_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device).unsqueeze(1)
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)

def evaluate():
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs = imgs.to(device)
            labels = labels.to(device).unsqueeze(1)
            outputs = model(imgs)
            preds = (torch.sigmoid(outputs) > 0.5).float()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return acc, prec, rec, f1

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [19]:
epochs = 10
for epoch in range(epochs):
    train_loss = train_one_epoch()
    acc, prec, rec, f1 = evaluate()

    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Acc={acc:.4f}, Precision={prec:.4f}, Recall={rec:.4f}, F1={f1:.4f}")

    if f1 > best_f1:
        best_f1 = f1
        best_model_wts = copy.deepcopy(model.state_dict())
        patience_counter = 0
        torch.save(model.state_dict(), "best_model.pth")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping!")
            break

model.load_state_dict(best_model_wts)


Epoch 1: Train Loss=0.5107, Acc=0.7897, Precision=0.7981, Recall=0.6506, F1=0.7168
Epoch 2: Train Loss=0.4491, Acc=0.8073, Precision=0.7814, Recall=0.7346, F1=0.7572
Epoch 3: Train Loss=0.4284, Acc=0.8188, Precision=0.8046, Recall=0.7358, F1=0.7687
Epoch 4: Train Loss=0.4073, Acc=0.8171, Precision=0.8583, Recall=0.6625, F1=0.7478
Epoch 5: Train Loss=0.3889, Acc=0.8141, Precision=0.7868, Recall=0.7486, F1=0.7673
Epoch 6: Train Loss=0.3761, Acc=0.8271, Precision=0.8371, Recall=0.7172, F1=0.7725
Epoch 7: Train Loss=0.3576, Acc=0.8238, Precision=0.8585, Recall=0.6817, F1=0.7600
Epoch 8: Train Loss=0.3476, Acc=0.8086, Precision=0.8639, Recall=0.6319, F1=0.7299
Epoch 9: Train Loss=0.3253, Acc=0.8165, Precision=0.7863, Recall=0.7575, F1=0.7716
Early stopping!


<All keys matched successfully>

In [ ]:
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

# Load model definition here
model = CNNTransformerModel().to(device)
model.load_state_dict(torch.load("best_model.pth"))
model.eval()

# Assuming val_dataset
test_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

all_preds = []
all_labels = []
all_paths = []

with torch.no_grad():
    for i, (images, labels) in enumerate(test_loader):
        images = images.to(device)
        labels = labels.to(device).unsqueeze(1)

        outputs = model(images)
        probs = torch.sigmoid(outputs)
        preds = (probs > 0.5).float()

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Metrics
acc = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds)
rec = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)

print(" Test Evaluation:")
print(f"Accuracy  : {acc:.4f}")
print(f"Precision : {prec:.4f}")
print(f"Recall    : {rec:.4f}")
print(f"F1 Score  : {f1:.4f}")



/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(



🧪 Test Evaluation:
Accuracy  : 0.8274
Precision : 0.8374
Recall    : 0.7175
F1 Score  : 0.7728
